# The Source Coding Theorem

## Information content of independent random variables

If $x$ and $y$ are independent, the following identities hold
$$h\left(x,y\right) = h\left(x\right) + h\left(y\right)$$
$$H\left(X,Y\right) = H\left(X\right) + H\left(Y\right)$$
I.e., entrop and the SHannon information content are additive for independent variables.

## Designing informative experiments

One important property of the entropy is the following: The entropy of an ensemble $X$ is biggest if all the outcomes have equal probabilitiy $p_i=1/|\mathcal{A}_X|$. In other words: The outcome of a random experiment is guaranteed to be most informative if the probability distribution over outcomes is uniform.

### Example: Customer support ticket categories

We want to get a better intuition of what this property means. Let us assume that a support team receives tickets in 4 categories:
* Billing
* Technical issue
* Account access
* Cancellation

If one category happens almost all the time, then each new ticket tells you little (low entropy). If all 4 categories are equally likely, each new ticket is more surprising/informative (high entropy).

Let us start by comparing the entropy of different examples of probability distributions:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from entropy_lab.measures.entropy import compute_entropy

labels = ["Billing", "Tech", "Access", "Cancel"]

distributions = {
    "Very skewed": np.array([0.85, 0.10, 0.03, 0.02]),
    "Moderately skewed": np.array([0.55, 0.20, 0.15, 0.10]),
    "Almost balanced": np.array([0.30, 0.25, 0.25, 0.20]),
    "Uniform (max entropy)": np.array([0.25, 0.25, 0.25, 0.25])
}

for name, p in distributions.items():
    H = compute_entropy(p)
    print(f"{name:22s} -> H = {H:.4f} bits")

As expected, we can observe that:
1. Entropy increases as the distribution becomes more balanced.
2. The uniform case gives the highest entropy.
   
We can visualize this property by generating a plot that moves gradually from a peaked distribution to a uniform one:

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 

from entropy_lab.measures.entropy import compute_entropy

p_skewed = np.array([0.85, 0.10, 0.03, 0.02])
p_uniform = np.array([0.25, 0.25, 0.25, 0.25])

alphas = np.linspace(0, 1, 101)
entropies = []

for a in alphas:
    # Interpolate between skewed and uniform
    p = (1 - a) * p_skewed + a * p_uniform
    H = compute_entropy(p, base=2.0)
    entropies.append(H)

plt.figure(figsize=(8, 4.5))
plt.plot(alphas, entropies, linewidth=2)
plt.axhline(np.log2(4), linestyle="--", linewidth=1, label="Max = log2(4) = 2 bits")
plt.xlabel("Balance level (0 = skewed, 1 = uniform)")
plt.ylabel("Entropy (bits)")
plt.title("Entropy is maximal when outcomes are equally likely")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## Data compression

We are now convinced that the Shannon information content of an outcome is a natural measure of its information content. In other words, improbable outcomes do convey more information than probable outcomes. Let us now consider how many bits are needed to describe the outcome of an experiment. 

If we can show that we can compress data from a particular source into a file of $L$ bits per source symbol and recover the data reliably, then we will say that the average information content of that source is at most $L$ bits per symbol.

### Example: Compression of IoT status messages

To examplify the idea of data compression, let us imagine an IoT sensor sending one status every minute:
* ```OK```: very common
* ```LOW_BATTERY```: rare
* ```SENSOR_ERROR```: very rare
* ```CALIBRATION```: occasional
* ```MAINTENANCE```: rare

We wish to encode these status messages and look for the optimal strategy. Encoding them with the same fixed length code is wasteful, since ```OK``` appears most of the time. Using Shannon's theory we understand that:
1. Frequent events carry less information
2. Rare events carry more information
3. So an efficient code should use:
    * short codes for common symbols
    * long codes for rare symbols

Let us try to simulate and visualize this.

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt

**Definition of a simple telemtry source** We model a sensor that sends one status message at a time. Most of the time everything is fine (```OK```), and rare events occur occasionally.

In [ ]:
# Possible status messages and their probabilities
symbols = np.array([
    "OK",
    "CALIBRATION",
    "LOW_BATTERY",
    "MAINTENANCE",
    "SENSOR_ERROR"
])

p = np.array([0.85, 0.07, 0.04, 0.03, 0.01], dtype=float)

**Computation of Shannon information content and entropy**

In [ ]:
from entropy_lab.measures.entropy import compute_entropy
from entropy_lab.measures.shannon import shannon_information

I = []
for proba in p: 
    I.append(shannon_information(proba))
H = compute_entropy(p)

print("Shannon information content per symbol (bits):")
for s, prob, info in zip(symbols, p, I):
    print(f"{s:15s}  p={prob:.2f}  I(x)={info:.3f} bits")
print(f"\nEntropy H(X) = {H:.3f} bits/message")

**Comparing with a fixed-length code** Jumping ahead in terms of theory, we can observe that, due to the fact that there is 5 possible messages, a fixed-length binary code would need 3 bits per message (theory comes later):

In [ ]:
fixed_length_bits = int(np.ceil(np.log2(len(symbols))))
print("Fixed-length code size:", fixed_length_bits, "bits/message")

**Definition of a simple prefix-free variable-length code** Without going into details, let us define an encoding in which: Short code for common events, longer codes for rare ones.

In [ ]:
# A simple prefix-free codebook
codebook = {
    "OK": "0",
    "CALIBRATION": "100",
    "LOW_BATTERY": "101",
    "MAINTENANCE": "110",
    "SENSOR_ERROR": "111"
}

lengths = np.array([len(codebook[s]) for s in symbols], dtype=int)
L_avg = np.sum(p * lengths)
print("Variable-length codebook:")
for s in symbols:
    print(f"{s:15s} -> {codebook[s]} (length {len(codebook[s])})")
print(f"\nAverage code length L = {L_avg:.3f} bits/message")

**Simulation** Now that we have a valid encoding, we can simulate a message stream and comparing the bit usage, in order to make the compression feel more concrete.

In [ ]:
# Seed
rng = np.random.default_rng(42)

# Simulate N messages from the source
N = 10000
samples = rng.choice(symbols, size=N, p=p)

# Count occurrences
unique, counts = np.unique(samples, return_counts=True)
count_dict = dict(zip(unique, counts))

# Total bits using fixed-length coding
total_bits_fixed = N * fixed_length_bits

# Total bits using variable-length coding
total_bits_variable = 0
for s in symbols:
    total_bits_variable += count_dict.get(s, 0) * len(codebook[s])

compression_ratio = total_bits_variable / total_bits_fixed
savings_percent = (1 - compression_ratio) * 100

print(f"Simulated messages: {N}")
print(f"Total bits (fixed-length):    {total_bits_fixed}")
print(f"Total bits (variable-length): {total_bits_variable}")
print(f"Compression ratio:            {compression_ratio:.3f}")
print(f"Bit savings:                  {savings_percent:.1f}%")

**Visualization** Finally, let us look at the results with the help of a graph.

In [ ]:
# Prepare values for the final comparison chart
labels_metrics = ["Entropy H(X)", "Avg code length L", "Fixed-length"]
values_metrics = [H, L_avg, fixed_length_bits]

# Sort symbols by probability (looks cleaner and tells the story better)
order = np.argsort(p)[::-1]
symbols_sorted = symbols[order]
p_sorted = p[order]
I = np.array(I, dtype=float)
I_sorted = I[order]

# A cleaner style without forcing a custom color palette
plt.rcParams.update({
    "font.size": 10,
    "axes.titlesize": 11,
    "axes.labelsize": 10,
    "figure.titlesize": 15
})

fig, axes = plt.subplots(1, 3, figsize=(17, 5.4))

# ---------------- Panel 1: Source probabilities ----------------
bars0 = axes[0].bar(symbols_sorted, p_sorted, edgecolor="black", linewidth=0.6)
axes[0].set_title("Source probabilities")
axes[0].set_ylabel("Probability")
axes[0].set_ylim(0, max(p_sorted) * 1.18)
axes[0].tick_params(axis="x", rotation=35)
axes[0].grid(axis="y", alpha=0.25, linestyle="--")

# Value labels
for rect, val in zip(bars0, p_sorted):
    axes[0].text(
        rect.get_x() + rect.get_width() / 2,
        rect.get_height() + 0.01,
        f"{val:.2f}",
        ha="center",
        va="bottom",
        fontsize=9
    )

# ---------------- Panel 2: Self-information ----------------
bars1 = axes[1].bar(symbols_sorted, I_sorted, edgecolor="black", linewidth=0.6)
axes[1].set_title("Shannon information per message")
axes[1].set_ylabel("Bits")
axes[1].set_ylim(0, max(I_sorted) * 1.18)
axes[1].tick_params(axis="x", rotation=35)
axes[1].grid(axis="y", alpha=0.25, linestyle="--")

# Value labels
for rect, val in zip(bars1, I_sorted):
    axes[1].text(
        rect.get_x() + rect.get_width() / 2,
        rect.get_height() + 0.06,
        f"{val:.2f}",
        ha="center",
        va="bottom",
        fontsize=9
    )

# ---------------- Panel 3: Compression comparison ----------------
bars2 = axes[2].bar(labels_metrics, values_metrics, edgecolor="black", linewidth=0.6)
axes[2].set_title("Compression benchmark")
axes[2].set_ylabel("Bits per message")
axes[2].set_ylim(0, max(values_metrics + [H + 1]) * 1.2)
axes[2].tick_params(axis="x", rotation=15)
axes[2].grid(axis="y", alpha=0.25, linestyle="--")

# Annotate bars with values
for rect, v in zip(bars2, values_metrics):
    axes[2].text(
        rect.get_x() + rect.get_width() / 2,
        rect.get_height() + 0.04,
        f"{v:.2f}",
        ha="center",
        va="bottom",
        fontsize=9
    )

# Shannon bound (cleaner placement)
axes[2].axhline(H, linestyle="--", linewidth=1)
axes[2].axhline(H + 1, linestyle="--", linewidth=1)
axes[2].text(2.35, H, "  H(X)", va="center", fontsize=9)
axes[2].text(2.35, H + 1, "  H(X)+1", va="center", fontsize=9)

# Light spine cleanup
for ax in axes:
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

fig.suptitle("Entropy Labs - IoT Status Message Compression", y=1.02)
fig.tight_layout()
plt.show()

**Interpretation**
* The source is highly skewed (mostly ```OK```), so it has low entropy.
* A fixed-length code wastes bits by giving every message 3 bits.
* A variable-length code exploits the non-uniformity and gets close to entropy.
* This is the core link between information theory and compression.